In [1]:
from lib import FrontierPipeline, FisherInformation

In [2]:
pipe=FrontierPipeline()
model, weights, embeddings=pipe(0)

/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)


[-0.03336096 -0.03336096 -0.03336096 ... -0.0333607  -0.0333607
 -0.0333607 ]
Epoch [10/10000], Loss: 0.7325
Epoch [20/10000], Loss: 0.7080
Epoch [30/10000], Loss: 0.6799
Epoch [40/10000], Loss: 0.6478
Epoch [50/10000], Loss: 0.6131
Epoch [60/10000], Loss: 0.5772
Epoch [70/10000], Loss: 0.5414
Epoch [80/10000], Loss: 0.5072
Epoch [90/10000], Loss: 0.4777
Epoch [100/10000], Loss: 0.4526
Epoch [110/10000], Loss: 0.4308
Epoch [120/10000], Loss: 0.4132
Epoch [130/10000], Loss: 0.4008
Epoch [140/10000], Loss: 0.3923
Epoch [150/10000], Loss: 0.3858
Epoch [160/10000], Loss: 0.3814
Epoch [170/10000], Loss: 0.3790
Epoch [180/10000], Loss: 0.3772
Epoch [190/10000], Loss: 0.3760
Epoch [200/10000], Loss: 0.3750
Epoch [210/10000], Loss: 0.3742
Epoch [220/10000], Loss: 0.3736
Epoch [230/10000], Loss: 0.3731
Epoch [240/10000], Loss: 0.3727
Epoch [250/10000], Loss: 0.3724
Epoch [260/10000], Loss: 0.3722
Epoch [270/10000], Loss: 0.3720
Epoch [280/10000], Loss: 0.3719
Epoch [290/10000], Loss: 0.3718
Epo

In [3]:
firing_rate, _=model(embeddings)

In [6]:
print(embeddings.shape)

torch.Size([900, 10])


In [7]:
import torch
def compute_fisher_information_hierarchical(model, embeddings, neuron_index=0, delta=0.0333):
    model.eval()  # Set model to evaluation mode
    
    # Ensure embeddings require gradients
    embeddings.requires_grad_(True)
    
    # Get firing rates and intermediate outputs
    firing_rate, _ = model(embeddings)  # Ignore x_0 for now
    firing_rate = firing_rate[:, neuron_index]  # Shape: (n_observations,)
    
    # Ensure firing rates are positive
    if torch.any(firing_rate <= 0):
        raise ValueError("Firing rates must be positive.")
    
    # Compute scaling factor if necessary
    scaled_firing_rate = firing_rate.detach() * delta  # Shape: (n_observations,)
    
    # Get parameters of the first linear layer
    params_theta1 = list(model.linear.parameters())
    num_params_theta1 = sum(p.numel() for p in params_theta1)
    print(f"Number of parameters in first layer: {num_params_theta1}")
    
    # Initialize gradient matrix
    gradients = torch.zeros((firing_rate.shape[0], num_params_theta1))
    
    # Compute gradients for each observation
    for i in range(firing_rate.shape[0]):
        model.zero_grad()
        # Compute scaled sqrt of firing rate
        sqrt_mu = torch.sqrt(scaled_firing_rate[i])
        # Backpropagate to get gradients
        (sqrt_mu * firing_rate[i]).backward(retain_graph=True)
        # Extract gradients
        grad_theta1 = []
        for param in params_theta1:
            grad_theta1.append(param.grad.detach().flatten())
        grad_theta1 = torch.cat(grad_theta1)
        gradients[i] = grad_theta1
    
    # Convert gradients to NumPy
    gradients_np = gradients.numpy()
    
    # Compute Fisher information matrix
    fisher_matrix = gradients_np.T @ gradients_np  # Shape: (num_params_theta1, num_params_theta1)
    
    print("Fisher information matrix shape:", fisher_matrix.shape)
    print("Fisher information matrix stats: min =", fisher_matrix.min(), 
          "max =", fisher_matrix.max(), "mean =", fisher_matrix.mean())
    
    # Check if Fisher matrix is positive semi-definite
    eigenvalues = np.linalg.eigvalsh(fisher_matrix)
    if np.any(eigenvalues < -1e-8):  # Allowing for small numerical errors
        print("Warning: Fisher information matrix has negative eigenvalues.")
    else:
        print("Fisher information matrix is positive semi-definite.")
    
    return fisher_matrix

compute_fisher_information_hierarchical(model, embeddings, neuron_index=0, delta=0.0333)

Number of parameters in first layer: 7690
Fisher information matrix shape: (7690, 7690)
Fisher information matrix stats: min = -569326.6 max = 884065.0 mean = 1.3651608


array([[    8.426675 ,    -6.328634 ,    -5.6791945, ...,    21.515673 ,
          -15.964235 ,   -11.584909 ],
       [   -6.328634 ,   159.29916  ,    91.56505  , ...,  -740.5652   ,
          549.4854   ,   398.7503   ],
       [   -5.6791945,    91.56505  ,    62.043594 , ...,  -431.72702  ,
          320.3333   ,   232.45941  ],
       ...,
       [   21.515673 ,  -740.5652   ,  -431.72702  , ...,  3546.4946   ,
        -2631.432    , -1909.5764   ],
       [  -15.964235 ,   549.4854   ,   320.3333   , ..., -2631.432    ,
         1952.4733   ,  1416.8702   ],
       [  -11.584909 ,   398.7503   ,   232.45941  , ..., -1909.5764   ,
         1416.8702   ,  1028.1938   ]], dtype=float32)

In [4]:
fisher_matrix=FisherInformation(model, embeddings, neuron_index=0, delta=0.0333)

Firing rate shape: torch.Size([900, 657])
take_neuron_0 shape: (900,)
take_neuron_0 stats: min = 27.26111 max = 55.472095 mean = 40.54113
Scaled firing rate stats: min = 0.907795 max = 1.8472209 mean = 1.3500196
Fisher information matrix shape: (10, 10)
Fisher information matrix stats: min = -324165.88 max = 472859.03 mean = 1154.0724


In [11]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt


def get_theta_hat_all_layers(model):
    """
    Extracts the parameters (weights and biases) from both linear layers of the model.

    Parameters:
    - model: Trained LNP model instance.

    Returns:
    - theta_hat: Estimated parameters as a numpy array.
    """
    with torch.no_grad():
        # First linear layer parameters
        weights1 = model.linear.weight.detach().numpy()  # Shape: (hidden_dim, input_dim)
        biases1 = model.linear.bias.detach().numpy()     # Shape: (hidden_dim,)

        # Second linear layer parameters
        weights2 = model.linear2.weight.detach().numpy()  # Shape: (n_neurons, hidden_dim)
        biases2 = model.linear2.bias.detach().numpy()     # Shape: (n_neurons,)

    # Flatten and concatenate all parameters into a single vector
    theta_hat = np.concatenate([
        weights1.flatten(),
        biases1,
        weights2.flatten(),
        biases2
    ])
    return theta_hat



def compute_fisher_information_all_layers(model, embeddings, neuron_index=0, delta=0.0333):
    model.eval()  # Set model to evaluation mode

    # Ensure embeddings require gradients
    embeddings.requires_grad_(True)

    # Get firing rates
    firing_rate, _ = model(embeddings)
    firing_rate = firing_rate[:, neuron_index]  # Shape: (n_observations,)

    # Ensure firing rates are positive
    if torch.any(firing_rate <= 0):
        raise ValueError("Firing rates must be positive.")

    # Compute scaling factor if necessary
    scaled_firing_rate = firing_rate.detach() * delta  # Shape: (n_observations,)

    # Get parameters of both linear layers
    params = list(model.linear.parameters()) + list(model.linear2.parameters())
    num_params = sum(p.numel() for p in params)
    print(f"Total number of parameters: {num_params}")

    # Initialize gradient matrix
    gradients = torch.zeros((firing_rate.shape[0], num_params))

    # Compute gradients for each observation
    for i in range(firing_rate.shape[0]):
        model.zero_grad()
        # Compute sqrt of scaled firing rate
        sqrt_mu = torch.sqrt(scaled_firing_rate[i])
        # Backpropagate to get gradients
        (sqrt_mu * firing_rate[i]).backward(retain_graph=True)
        # Extract gradients
        grad_params = []
        for param in params:
            grad_params.append(param.grad.detach().flatten())
        grad_params = torch.cat(grad_params)
        gradients[i] = grad_params

    # Convert gradients to NumPy
    gradients_np = gradients.numpy()

    # Compute Fisher information matrix
    fisher_matrix = gradients_np.T @ gradients_np  # Shape: (num_params, num_params)

    print("Fisher information matrix shape:", fisher_matrix.shape)
    print("Fisher information matrix stats: min =", fisher_matrix.min(),
          "max =", fisher_matrix.max(), "mean =", fisher_matrix.mean())

    # Check if Fisher matrix is positive semi-definite
    eigenvalues = np.linalg.eigvalsh(fisher_matrix)
    if np.any(eigenvalues < -1e-8):  # Allowing for small numerical errors
        print("Warning: Fisher information matrix has negative eigenvalues.")
    else:
        print("Fisher information matrix is positive semi-definite.")

    return fisher_matrix

def test_coefficients_significance(theta_hat, fisher_matrix, confidence_level=0.95):
    """
    Test the statistical significance of regression coefficients using the Fisher information matrix.

    Parameters:
    - theta_hat (numpy array): Estimated parameters (shape: p,)
    - fisher_matrix (numpy array): Fisher information matrix (shape: p, p)
    - confidence_level (float): Desired confidence level (default: 0.95)

    Returns:
    - results (list of dict): Each dict contains parameter index, estimate, standard error, z-score, p-value, and significance
    """
    # Step 1: Compute Covariance Matrix
    try:
        cov_matrix = np.linalg.inv(fisher_matrix)
    except np.linalg.LinAlgError:
        # Regularize the Fisher information matrix if it's singular
        print("Fisher information matrix is singular or ill-conditioned. Adding a small value to the diagonal for regularization.")
        regularization_term = 1e-8 * np.eye(fisher_matrix.shape[0])
        cov_matrix = np.linalg.inv(fisher_matrix + regularization_term)

    # Step 2: Compute Standard Errors
    standard_errors = np.sqrt(np.diag(cov_matrix))

    # Avoid division by zero in standard errors
    standard_errors = np.where(standard_errors == 0, 1e-8, standard_errors)

    # Step 3: Compute Z-scores
    z_scores = theta_hat / standard_errors

    # Step 4: Compute P-values
    p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))

    # Step 5: Assess Significance
    alpha = 1 - confidence_level
    significant = p_values < alpha

    # Compile Results
    results = []
    for idx, (theta, se, z, p, sig) in enumerate(zip(theta_hat, standard_errors, z_scores, p_values, significant)):
        result = {
            'Parameter Index': idx,
            'Estimate': theta,
            'Standard Error': se,
            'Z-score': z,
            'P-value': p,
            f'Significant at {int(confidence_level*100)}%': sig
        }
        results.append(result)

    return results




theta_hat=get_theta_hat_all_layers(model)
fisher_matrix=compute_fisher_information_all_layers(model, embeddings, neuron_index=0, delta=0.0333)
test_coefficients_significance(theta_hat, fisher_matrix, confidence_level=0.95)

Total number of parameters: 14917
Fisher information matrix shape: (14917, 14917)
Fisher information matrix stats: min = -565847200.0 max = 827506750.0 mean = 2.0512176
Fisher information matrix is singular or ill-conditioned. Adding a small value to the diagonal for regularization.


/tmp/ipykernel_1677307/2076510044.py:116: RuntimeWarning: invalid value encountered in sqrt
  standard_errors = np.sqrt(np.diag(cov_matrix))


[{'Parameter Index': 0,
  'Estimate': 0.011454333,
  'Standard Error': 178.11867070810135,
  'Z-score': 6.430731025764665e-05,
  'P-value': 0.9999486901900341,
  'Significant at 95%': False},
 {'Parameter Index': 1,
  'Estimate': -0.0063936855,
  'Standard Error': 28.7083370623135,
  'Z-score': -0.00022271180171072103,
  'P-value': 0.9998223016933754,
  'Significant at 95%': False},
 {'Parameter Index': 2,
  'Estimate': 0.000749979,
  'Standard Error': 32.22849884026737,
  'Z-score': 2.32706772124496e-05,
  'P-value': 0.9999814326859344,
  'Significant at 95%': False},
 {'Parameter Index': 3,
  'Estimate': -0.040409014,
  'Standard Error': 43.411008444708294,
  'Z-score': -0.0009308471532152446,
  'P-value': 0.9992572915352389,
  'Significant at 95%': False},
 {'Parameter Index': 4,
  'Estimate': -0.044286657,
  'Standard Error': 87.3998521220371,
  'Z-score': -0.0005067131814241511,
  'P-value': 0.9995957013930876,
  'Significant at 95%': False},
 {'Parameter Index': 5,
  'Estimate': 

In [12]:
def test_coefficients_significance(theta_hat, fisher_matrix, confidence_level=0.95):
    """
    Test the statistical significance of regression coefficients using the Fisher information matrix.

    Parameters:
    - theta_hat (numpy array): Estimated parameters (shape: p,)
    - fisher_matrix (numpy array): Fisher information matrix (shape: p, p)
    - confidence_level (float): Desired confidence level (default: 0.95)

    Returns:
    - results (list of dict): Each dict contains parameter index, estimate, standard error, z-score, p-value, and significance
    """
    # Step 1: Compute Covariance Matrix
    try:
        cov_matrix = np.linalg.inv(fisher_matrix)
    except np.linalg.LinAlgError:
        # Regularize the Fisher information matrix if it's singular
        print("Fisher information matrix is singular or ill-conditioned. Adding a small value to the diagonal for regularization.")
        regularization_term = 1e-8 * np.eye(fisher_matrix.shape[0])
        cov_matrix = np.linalg.inv(fisher_matrix + regularization_term)

    # Step 2: Compute Standard Errors
    standard_errors = np.sqrt(np.diag(cov_matrix))

    # Avoid division by zero in standard errors
    standard_errors = np.where(standard_errors == 0, 1e-8, standard_errors)

    # Step 3: Compute Z-scores
    z_scores = theta_hat / standard_errors

    # Step 4: Compute P-values
    p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))

    # Step 5: Assess Significance
    alpha = 1 - confidence_level
    significant = p_values < alpha

    

    # Compile Results
    results = []
    for idx, (theta, se, z, p, sig) in enumerate(zip(theta_hat, standard_errors, z_scores, p_values, significant)):
        result = {
            'Parameter Index': idx,
            'Estimate': theta,
            'Standard Error': se,
            'Z-score': z,
            'P-value': p,
            f'Significant at {int(confidence_level*100)}%': sig
        }
        results.append(result)

    return results

results=test_coefficients_significance(theta_hat, fisher_matrix, confidence_level=0.95)

Fisher information matrix is singular or ill-conditioned. Adding a small value to the diagonal for regularization.


/tmp/ipykernel_1677307/1363067821.py:23: RuntimeWarning: invalid value encountered in sqrt
  standard_errors = np.sqrt(np.diag(cov_matrix))


In [14]:
sig=[r[f'Significant at {int(0.95*100)}%'] for r in results]

np.where(sig==True)


/tmp/ipykernel_1677307/871683672.py:3: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  np.where(sig==True)


(array([], dtype=int64),)